In [1]:
# HYBRID LSTM MODEL - PRODUCTION PREDICTION PIPELINE
# Purpose: Back-testing and Real-time Prediction for EMI Payment Default Risk
import pandas as pd
import numpy as np
import joblib
import glob
import os
import sqlalchemy
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# ===================================================================================
# GLOBAL CONFIGURATION
# ===================================================================================

# Database Configuration
DB_CONFIG = {
    'host': '10.192.5.43',
    'port': 5432,
    'database': 'postgres',
    'user': 'ml_db',
    'password': 'pass%401234'
}

# Data Configuration
LOS_TABLE_NAME = "Ashirvad"
LOS_START_DATE = '2024-01-01'
LOS_END_DATE = '2024-12-31'
NEW_LMS_FOLDER_PATH = "TestData_Oct2025prediction"

# Model Configuration
MAX_SEQUENCE_LENGTH = 8
ROLLING_WINDOW_SIZE = 3
GRACE_PERIOD_DAYS = 2
PAID_PERCENTAGE_THRESHOLD = 0.90

# Validation Configuration
TARGET_BACKTEST_EMI = 1  # Predict EMI N using data up to EMI N-1

# Status Mapping
TWO_CLASS_STATUS_MAP = {0: 'Paid', 1: 'Not Paid'}
FLAG_ORDER = {'A': 4, 'B': 3, 'C': 2, 'D': 1}

# Feature Definitions (MUST MATCH TRAINING EXACTLY)
SEQUENTIAL_COLS_NUMERICAL = [
    'INSTALLMENT_NO', 'INSTALLMENT_AMOUNT', 'DAYS_LATE', 'DAYS_BETWEEN_DUE_DATES',
    'PAID_RATIO', 'DELTA_DAYS_LATE', 'PAYMENT_SCORE', 'COMPOSITE_RISK',
    'RECENT_PAYMENT_SCORE', 'PAYMENT_SCORE_RANK', 'IS_UNPAID', 'CURRENT_EMI_BEHAVIOR_LABEL'
]

STATIC_COLS_NUMERICAL = ['TOTAL_INCOME', 'TOTAL_EXPENSE', 'LOAN_AMOUNT', 'AGE', 'CYCLE']
STATIC_COLS_OHE = ['MARITAL_STATUS_NAME', 'STATE_NAME', 'LOAN_SCHEDULE_TYPE']
STATIC_EMBEDDING_COLS = ['OCCUPATION_NAME_ENCODED', 'LOAN_PURPOSE_ENCODED']

In [3]:
def load_artifacts():
    """
    Load all required model artifacts. 
    Strict Version: Will raise a FileNotFoundError and stop execution if ANY file is missing.
    """
    # 1. Define required files
    artifact_paths = {
        'preprocessor': 'preprocessor.pkl',
        'model': 'hybrid_lstm_model.h5',
        'mappings': 'embedding_category_mappings.pkl'
    }

    # 2. Pre-check existence for clear reporting
    missing_files = [path for path in artifact_paths.values() if not os.path.exists(path)]
    
    if missing_files:
        print("\n" + "!" * 60)
        print("❌ CRITICAL ERROR: MISSION-CRITICAL ARTIFACTS MISSING")
        print(f"The following required files were not found: {missing_files}")
        print("Pipeline execution halted to prevent incorrect predictions.")
        print("!" * 60 + "\n")
        # Explicitly raise error to stop the script
        raise FileNotFoundError(f"Missing artifacts: {missing_files}")

    # 3. Load artifacts strictly
    try:
        preprocessor = joblib.load(artifact_paths['preprocessor'])
        model = load_model(artifact_paths['model'], compile=False)
        category_mappings = joblib.load(artifact_paths['mappings'])
        
        print("✅ Success: Preprocessor, Model, and Mappings loaded.")
        return preprocessor, model, category_mappings
        
    except Exception as e:
        print(f"❌ ERROR: Artifacts present but failed to load. Check for corruption or version mismatch: {e}")
        raise

In [4]:
# ===================================================================================
# PART 1: LOS DATA LOADING AND PREPROCESSING
# ===================================================================================

def load_and_preprocess_los():
    """
    Loads LOS data from PostgreSQL and applies ALL preprocessing steps 
    exactly as done in training (1_los_data_prep.ipynb).
    """
    conn_string = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
    
    try:
        engine = sqlalchemy.create_engine(conn_string)
        print(f"--- Loading LOS Data from PostgreSQL Table: {LOS_TABLE_NAME} ---")
        
        query = f"""
        SELECT *
        FROM "{LOS_TABLE_NAME}"
        WHERE "LOAN_DATE" BETWEEN '{LOS_START_DATE}' AND '{LOS_END_DATE}';
        """
        df = pd.read_sql_query(query, con=engine)
        print(f"✅ LOS data loaded. Shape: {df.shape}")
        
    except Exception as e:
        print(f"❌ Database connection or query failed: {e}")
        raise
    
    # === EXACT TRAINING PREPROCESSING ===
    
    print("--- Applying LOS Preprocessing (Matching Training) ---")
    
    # 1. Convert PIN_CODE to string
    if 'PIN_CODE' in df.columns:
        df['PIN_CODE'] = df['PIN_CODE'].astype(str)
    
    # 2. Drop columns (matching training)
    columns_to_drop = [
        'CUSTOMER_ID', 'CUSTOMER_NAME', 'BRANCH_ID', 'TEMP_CUST_ID',
        'PHONE1', 'PHONE2', 'HOUSE_NAME', 'LOCALITY', 'STREET',
        'ALT_HOUSE_NAME', 'ALT_LOCALITY', 'ALT_STREET', 'CENTER_ID',
        'ALT_PIN_CODE', 'MARITAL_STATUS', 'LOAN_STATUS',
        'LOAN_STATUS_DESC', 'CLS_DT', 'CIBIL_ID', 'NPA_FLAG',
        'NPA_FROM_DATE', 'NPA_TO_DATE', 'NPA_STATUS',
        'OCCUPATION_ID', 'APPLICATION_ID', 'TENURE_in_months',
        'emi_paid', 'loan_paid_percentage', 'NPA_STATUS_UPDATED',
        'NPA_STATUS_UPDATED_1', 'NPA_STATUS_UPDATED_2', 'YEAR'
    ]
    df.drop(columns=columns_to_drop, inplace=True, errors='ignore')
    
    # 3. Drop CIBIL_SCORE and CUSTOMER_GRADING_SCORE
    df.drop(columns=['CIBIL_SCORE', 'CUSTOMER_GRADING_SCORE'], inplace=True, errors='ignore')
    
    # 4. Calculate AGE
    df['DATE_OF_BIRTH'] = pd.to_datetime(df['DATE_OF_BIRTH'], errors='coerce')
    df['LOAN_DATE'] = pd.to_datetime(df['LOAN_DATE'], errors='coerce')
    df['AGE'] = (df['LOAN_DATE'] - df['DATE_OF_BIRTH']).dt.days // 365.25
    df.drop(columns=['DATE_OF_BIRTH'], inplace=True)
    
    # 5. Filter CUSTOMER_FLAG (remove 'X' records)
    df = df[df['CUSTOMER_FLAG'] != 'X'].copy()
    
    # 6. Drop high-cardinality columns
    df.drop(columns=['BRANCH_NAME', 'CENTER_NAME'], errors='ignore', inplace=True)
    
    # 7. Impute missing values in numerical columns (USING TRAINING MEDIANS)
    imputation_values = {
        'TOTAL_INCOME': 20000.0,
        'TOTAL_EXPENSE': 1760.0,
        'AGE':  38.0
    }
    
    for col, median_val in imputation_values.items():
        if col in df.columns and df[col].isnull().sum() > 0:
            # Use training median if available, otherwise compute from current data
            #median_val = df[col].median() if col == 'AGE' else default_val
            df[col] = df[col].fillna(median_val)
            print(f"   Imputed {col} with value: {median_val:.2f}")
    
    # 8. Impute OCCUPATION_NAME with 'UNKNOWN'
    if 'OCCUPATION_NAME' in df.columns:
        df['OCCUPATION_NAME'] = df['OCCUPATION_NAME'].fillna('UNKNOWN')
    
    # 9. Consolidate MARITAL_STATUS_NAME
    if 'MARITAL_STATUS_NAME' in df.columns:
        df['MARITAL_STATUS_NAME'] = df['MARITAL_STATUS_NAME'].replace(
            ['UNMARRIED', 'SINGLE'], 'UNMARRIED/SINGLE')
    
    print(f"✅ LOS Preprocessing Complete. Shape: {df.shape}")
    return df

# ===================================================================================
# PART 2: LMS DATA LOADING AND FEATURE ENGINEERING
# ===================================================================================

def load_and_engineer_lms():
    """
    Loads LMS data and applies ALL feature engineering steps 
    exactly as done in training (2_lms_data_prep.ipynb).
    """
    all_lms_dfs = []
    lms_files = sorted(glob.glob(os.path.join(NEW_LMS_FOLDER_PATH, 'test_*.xlsx')))
    
    if not lms_files:
        raise FileNotFoundError(f"No Excel files found matching 'test_*.xlsx' in {NEW_LMS_FOLDER_PATH}.")
    
    print(f"--- Loading {len(lms_files)} LMS Excel file(s) ---")
    for file_name in lms_files:
        try:
            df = pd.read_excel(file_name)
            all_lms_dfs.append(df)
            print(f"   ✅ Loaded: {os.path.basename(file_name)}")
        except Exception as e:
            print(f"   ❌ Error loading {os.path.basename(file_name)}: {e}")
    
    df_lms = pd.concat(all_lms_dfs, ignore_index=True)
    
    # === EXACT TRAINING FEATURE ENGINEERING ===
    
    print("--- Applying LMS Feature Engineering ---")
    
    # Core Cleaning
    df_lms['DUE_DATE'] = pd.to_datetime(df_lms['DUE_DATE'], errors='coerce')
    df_lms['PAID_DT'] = pd.to_datetime(df_lms['PAID_DT'], errors='coerce')
    df_lms.loc[df_lms['STATUS'] == 1, 'PAID_DT'] = pd.NaT
    df_lms = df_lms.sort_values(by=['LOAN_ID', 'INSTALLMENT_NO']).reset_index(drop=True)
    
    # Remove invalid records
    df_lms = df_lms.loc[~((df_lms['PAID_DT'].isnull()) & (df_lms['STATUS'] != 1))].reset_index(drop=True)
    
    # Core Features
    df_lms['DAYS_LATE'] = (df_lms['PAID_DT'] - df_lms['DUE_DATE']).dt.days
    df_lms['DAYS_BETWEEN_DUE_DATES'] = df_lms.groupby('LOAN_ID')['DUE_DATE'].diff().dt.days
    
    # Repayment Schedule Category
    df_lms['REPAYMENT_SCHEDULE_CAT'] = np.select(
        [
            df_lms['DAYS_BETWEEN_DUE_DATES'].isnull(),
            df_lms['DAYS_BETWEEN_DUE_DATES'].isin([28, 29, 30, 31]),
            df_lms['DAYS_BETWEEN_DUE_DATES'] == 7,
            df_lms['DAYS_BETWEEN_DUE_DATES'] == 14,
            df_lms['DAYS_BETWEEN_DUE_DATES'] == 56
        ],
        ['Initial', 'Monthly', 'Weekly', 'Bi-Weekly', 'Bi-Monthly'],
        default='Other'
    )
    
    # Loan Schedule Type
    has_weekly = df_lms.groupby('LOAN_ID')['REPAYMENT_SCHEDULE_CAT'].transform(
        lambda x: 'Weekly' in x.unique())
    has_monthly = df_lms.groupby('LOAN_ID')['REPAYMENT_SCHEDULE_CAT'].transform(
        lambda x: 'Monthly' in x.unique())
    mode_schedule = df_lms.groupby('LOAN_ID')['REPAYMENT_SCHEDULE_CAT'].transform(
        lambda x: x.mode()[0] if not x.mode().empty else 'Initial')
    df_lms['LOAN_SCHEDULE_TYPE'] = np.where((has_weekly) & (has_monthly), 'Hybrid', mode_schedule)
    
    # Payment Features
    df_lms['IS_UNPAID'] = np.where(df_lms['STATUS'] == 1, 1, 0)
    df_lms['IS_DAYS_LATE_MISSING'] = df_lms['DAYS_LATE'].isna().astype(int)
    df_lms['DAYS_LATE'] = df_lms['DAYS_LATE'].fillna(0)
    df_lms['PAID_RATIO'] = (df_lms['PAID_AMOUNT'] / df_lms['INSTALLMENT_AMOUNT']).clip(upper=1.0)
    df_lms['DELTA_DAYS_LATE'] = df_lms.groupby('LOAN_ID')['DAYS_LATE'].diff().fillna(0)
    
    # CRITICAL: Zero out DELTA_DAYS_LATE for unpaid EMIs (matching training)
    df_lms.loc[df_lms['IS_UNPAID'] == 1, 'DELTA_DAYS_LATE'] = 0
    
    # Current EMI Behavior Label
    conditions_behavior = [
        (df_lms['STATUS'] == 1) |
        ((df_lms['STATUS'] == 0) & (df_lms['DAYS_LATE'] > GRACE_PERIOD_DAYS)) |
        ((df_lms['STATUS'] == 2) & (
            (df_lms['DAYS_LATE'] > GRACE_PERIOD_DAYS) |
            (df_lms['PAID_AMOUNT'] / df_lms['INSTALLMENT_AMOUNT'] < PAID_PERCENTAGE_THRESHOLD)
        )),
        (df_lms['STATUS'] == 0) & (df_lms['DAYS_LATE'] <= GRACE_PERIOD_DAYS),
        (df_lms['STATUS'] == 2) & (df_lms['DAYS_LATE'] <= GRACE_PERIOD_DAYS) &
        (df_lms['PAID_AMOUNT'] / df_lms['INSTALLMENT_AMOUNT'] >= PAID_PERCENTAGE_THRESHOLD)
    ]
    choices_behavior = [1, 0, 0]
    df_lms['CURRENT_EMI_BEHAVIOR_LABEL'] = np.select(
        conditions_behavior, choices_behavior, default=-1).astype(np.int8)
    
    # Composite Risk and Payment Score
    df_lms['REMAINING_EMI_RATIO'] = 1 - df_lms["PAID_RATIO"]
    df_lms['COMPOSITE_RISK'] = df_lms['DAYS_LATE'] + (df_lms['REMAINING_EMI_RATIO'] * 10)
    
    conditions_score = [
        df_lms['IS_UNPAID'] == 1,
        (df_lms['CURRENT_EMI_BEHAVIOR_LABEL'] == 1) & (df_lms['IS_UNPAID'] == 0),
        (df_lms['CURRENT_EMI_BEHAVIOR_LABEL'] == 0) & (df_lms['DAYS_LATE'] > 0) &
        (df_lms['DAYS_LATE'] <= GRACE_PERIOD_DAYS) & (df_lms['IS_UNPAID'] == 0),
        (df_lms['CURRENT_EMI_BEHAVIOR_LABEL'] == 0) & (df_lms['DAYS_LATE'] <= 0) &
        (df_lms['IS_UNPAID'] == 0)
    ]
    
    choices_score = [
        -100,
        np.maximum(0.0, np.minimum(0.30, 0.30 - (df_lms['COMPOSITE_RISK'] * 0.03))),
        1.0 / (1 + df_lms['COMPOSITE_RISK']),
        1.5 + (np.abs(df_lms['DAYS_LATE']) / 10)
    ]
    df_lms['PAYMENT_SCORE'] = np.select(conditions_score, choices_score, default=0).astype(np.float32)
    
    rank_choices = [4, 3, 2, 1]
    df_lms['PAYMENT_SCORE_RANK'] = np.select(conditions_score, rank_choices).astype(np.int8)
    
    # === VALIDATION MODE: Extract actual targets before trimming ===
    actual_targets = df_lms.groupby('LOAN_ID').tail(TARGET_BACKTEST_EMI).copy()
    actual_targets = actual_targets[['LOAN_ID', 'CURRENT_EMI_BEHAVIOR_LABEL']].rename(
        columns={'CURRENT_EMI_BEHAVIOR_LABEL': 'ACTUAL_NEXT_EMI_ISSUE'}
    )
    
    print(f"✅ LMS Feature Engineering Complete. Shape: {df_lms.shape}")
    return df_lms, actual_targets

# ===================================================================================
# PART 3: DATA MERGING AND FINAL PREPARATION
# ===================================================================================

def merge_and_finalize_data(df_los, df_lms_full, df_actual_targets, category_mappings):
    """
    Merges LOS and LMS data, applies final feature engineering,
    and performs back-test trimming exactly as in training (3_hybrid_data_prep_optimized.ipynb).
    """
    
    # 1. VALIDATION MODE: Trim the last EMI (EMI N) from sequence
    print(f"\n--- Back-testing Mode: Trimming last {TARGET_BACKTEST_EMI} EMI(s) ---")
    indices_to_drop = df_lms_full.groupby('LOAN_ID').tail(TARGET_BACKTEST_EMI).index
    df_lms = df_lms_full.drop(indices_to_drop).reset_index(drop=True)
    print(f"   LMS records after trimming: {df_lms.shape[0]}")
    
    # 2. Rename LMS columns to avoid conflicts
    RENAME_MAP_LMS = {
        'LOAN_AMOUNT': 'LOAN_AMOUNT_LMS',
        'TENURE': 'TENURE_LMS',
        'INTEREST_RATE': 'INTEREST_RATE_LMS'
    }
    df_lms.rename(columns=RENAME_MAP_LMS, inplace=True)
    
    # 3. Rename LOS columns
    RENAME_MAP_LOS = {
        'LOAN_AMOUNT': 'LOAN_AMOUNT_STATIC',
        'TENURE': 'TENURE_STATIC',
        'INTEREST_RATE': 'INTEREST_RATE_STATIC'
    }
    df_los.rename(columns=RENAME_MAP_LOS, inplace=True)
    
    # 4. Merge
    df_combined = pd.merge(df_los, df_lms, on='LOAN_ID', how='inner', suffixes=('_static', '_lms'))
    print(f"   Combined shape after merge: {df_combined.shape}")
    
    # 5. Clean up column names
    df_combined.rename(columns={
        "LOAN_DATE_static": "LOAN_DATE",
        "LOAN_AMOUNT_STATIC": "LOAN_AMOUNT",
        "TENURE_STATIC": "TENURE",
        "INTEREST_RATE_STATIC": "INTEREST_RATE"
    }, inplace=True)
    
    # 6. Drop unnecessary columns
    cols_to_drop = [
        'LOAN_DATE_lms', 'LOAN_AMOUNT_lms', 'TENURE_lms', 'INTEREST_RATE_lms',
        'DISBURSED_AMOUNT', 'PIN_CODE', 'IS_DAYS_LATE_MISSING', 'STATUS',
        'PAID_DT', 'PAID_AMOUNT', 'REMAINING_EMI_RATIO', 'TENURE', 'INTEREST_RATE'
    ]
    df_combined.drop(columns=[c for c in cols_to_drop if c in df_combined.columns],
                     errors='ignore', inplace=True)
    
    # 7. Impute DAYS_BETWEEN_DUE_DATES
    df_combined["DAYS_BETWEEN_DUE_DATES"].fillna(0, inplace=True)
    
    # 8. Rolling Feature (RECENT_PAYMENT_SCORE) - WITH PROPER SHIFTING
    print("   Creating RECENT_PAYMENT_SCORE (rolling window feature)...")
    df_combined['RECENT_PAYMENT_SCORE'] = df_combined.groupby('LOAN_ID')['PAYMENT_SCORE'].rolling(
        window=ROLLING_WINDOW_SIZE, min_periods=1
    ).mean().reset_index(level=0, drop=True).astype(np.float32)
    
    # CRITICAL: Shift to prevent leakage
    df_combined['RECENT_PAYMENT_SCORE'] = df_combined.groupby('LOAN_ID')['RECENT_PAYMENT_SCORE'].shift(1)
    
    # Use TRAINING mean for imputation
    overall_mean_score = -39.098625  # From training
    df_combined['RECENT_PAYMENT_SCORE'].fillna(overall_mean_score, inplace=True)
    
    # 9. Custom Encoding
    print("   Applying custom encoding for CUSTOMER_FLAG, OCCUPATION_NAME, LOAN_PURPOSE...")
    df_combined['CUSTOMER_FLAG_ENCODED'] = df_combined['CUSTOMER_FLAG'].map(FLAG_ORDER).fillna(0).astype(np.int8)
    
    # Use training category mappings for consistent encoding
    if category_mappings['occupation'] and category_mappings['purpose']:
        occupation_to_code = {v: k+1 for k, v in category_mappings['occupation'].items()}
        purpose_to_code = {v: k+1 for k, v in category_mappings['purpose'].items()}
        
        df_combined['OCCUPATION_NAME_ENCODED'] = df_combined['OCCUPATION_NAME'].astype(str).map(
            occupation_to_code).fillna(0).astype(np.int16)
        df_combined['LOAN_PURPOSE_ENCODED'] = df_combined['LOAN_PURPOSE'].astype(str).map(
            purpose_to_code).fillna(0).astype(np.int16)
        
        print(f"      OCCUPATION_NAME: {df_combined['OCCUPATION_NAME_ENCODED'].nunique()} unique codes")
        print(f"      LOAN_PURPOSE: {df_combined['LOAN_PURPOSE_ENCODED'].nunique()} unique codes")
    else:
        print("   ⚠️ WARNING: Using fallback encoding (may be inconsistent)")
    
    df_combined.drop(columns=['CUSTOMER_FLAG', 'OCCUPATION_NAME', 'LOAN_PURPOSE', 'LOAN_DATE'],
                     errors='ignore', inplace=True)
    
    # 10. OHE Sequential Categorical Feature
    print("   One-hot encoding REPAYMENT_SCHEDULE_CAT...")
    df_combined = pd.get_dummies(df_combined, columns=['REPAYMENT_SCHEDULE_CAT'], prefix='REPAYMENT_CAT')
    
    # 11. Filter for the latest sequence for prediction
    print("   Filtering to keep only necessary sequence (last MAX_SEQUENCE_LENGTH EMIs)...")
    df_combined['SEQUENCE_COUNT'] = df_combined.groupby('LOAN_ID').cumcount() + 1
    df_combined['REVERSE_SEQUENCE_COUNT'] = df_combined.groupby('LOAN_ID')['SEQUENCE_COUNT'].transform(
        'max') - df_combined['SEQUENCE_COUNT']
    df_combined_filtered = df_combined[df_combined['REVERSE_SEQUENCE_COUNT'] < MAX_SEQUENCE_LENGTH].copy()
    df_combined_filtered.drop(columns=['SEQUENCE_COUNT', 'REVERSE_SEQUENCE_COUNT'], inplace=True)
    
    # 12. Merge Actual Target for Validation
    df_combined_filtered = pd.merge(df_combined_filtered, df_actual_targets, on='LOAN_ID', how='left')
    df_combined_filtered['ACTUAL_NEXT_EMI_ISSUE'] = df_combined_filtered.groupby(
        'LOAN_ID')['ACTUAL_NEXT_EMI_ISSUE'].transform('max')
    df_combined_filtered.dropna(subset=['ACTUAL_NEXT_EMI_ISSUE'], inplace=True)
    
    print(f"✅ Data preparation complete. Final shape: {df_combined_filtered.shape}")
    print(f"   Unique loans: {df_combined_filtered['LOAN_ID'].nunique()}")
    
    return df_combined_filtered

# ===================================================================================
# PART 4: PREPROCESSING AND PREDICTION
# ===================================================================================

def preprocess_and_predict(df_full, preprocessor, model):
    """
    Applies preprocessor, reshapes sequences, and runs inference.
    """
    
    print("\n--- Step: Preprocessing and Prediction ---")

    REPAYMENT_CAT_OHE_COLS = [c for c in df_full.columns if c.startswith('REPAYMENT_CAT_')]
    
    # Add CUSTOMER_FLAG_ENCODED to static numerical features
    STATIC_COLS_NUMERICAL_WITH_FLAG = STATIC_COLS_NUMERICAL + ['CUSTOMER_FLAG_ENCODED']
    
    # Reconstruct NUMERICAL_FEATURES_FINAL exactly as in training
    NUMERICAL_FEATURES_FINAL = SEQUENTIAL_COLS_NUMERICAL + \
                               STATIC_COLS_NUMERICAL_WITH_FLAG + \
                               REPAYMENT_CAT_OHE_COLS
    
    print(f"   Reconstructed NUMERICAL_FEATURES_FINAL with {len(NUMERICAL_FEATURES_FINAL)} features")
    print(f"      - Sequential: {len(SEQUENTIAL_COLS_NUMERICAL)}")
    print(f"      - Static Numerical: {len(STATIC_COLS_NUMERICAL_WITH_FLAG)}")
    print(f"      - Repayment OHE: {len(REPAYMENT_CAT_OHE_COLS)}")
    
    # Get expected features from preprocessor
    expected_features = list(preprocessor.feature_names_in_)
    
    # Align DataFrame columns with expected features (safe approach)
    df_aligned = df_full.reindex(columns=expected_features, fill_value=0)
    print(f"   DataFrame aligned to {len(expected_features)} expected features")
    
    # Apply ColumnTransformer
    print("   Applying StandardScaler and OneHotEncoder...")
    X_scaled_ohe = preprocessor.transform(df_aligned)
    ALL_FINAL_COLS = list(preprocessor.get_feature_names_out())

    # Reconstruct DataFrame
    X_predict_df = pd.DataFrame(X_scaled_ohe, columns=ALL_FINAL_COLS, index=df_full.index)
    
    # Add back non-transformed columns
    X_predict_df['LOAN_ID'] = df_full['LOAN_ID'].values
    X_predict_df['ACTUAL_NEXT_EMI_ISSUE'] = df_full['ACTUAL_NEXT_EMI_ISSUE'].values
    for col in STATIC_EMBEDDING_COLS:
        X_predict_df[col] = df_full[col].values
    # ============================================================
    # ✅ STEP 5: CORRECTED - Define input feature sets
    # ============================================================
    
    # Build a set of exact feature names we want to include/exclude
    # This prevents partial string matching bugs
    
    # 5A. LSTM Features (Sequential + Repayment OHE)
    lstm_sequential_features = [f"num__{col}" for col in SEQUENTIAL_COLS_NUMERICAL]
    lstm_repayment_features = [c for c in ALL_FINAL_COLS if c.startswith('num__REPAYMENT_CAT_')]
    LSTM_INPUT_COLS_FINAL = lstm_sequential_features + lstm_repayment_features
    
    # 5B. Static Dense Features (Numerical + OHE)
    # ✅ CORRECTED: Use exact column name matching to avoid substring bugs
    static_numerical_features = [f"num__{col}" for col in STATIC_COLS_NUMERICAL_WITH_FLAG]
    
    # Get OHE features - these have 'cat__' prefix from the ColumnTransformer
    static_ohe_features = [c for c in ALL_FINAL_COLS 
                          if any(c.startswith(f'cat__{base}') for base in STATIC_COLS_OHE)]
    
    STATIC_DENSE_COLS_FINAL = static_numerical_features + static_ohe_features
    # ============================================================
    # ✅ STEP 6: CRITICAL VALIDATION - Verify all features exist
    # ============================================================
    
    # Check if all expected features are in the transformed data
    missing_lstm = [f for f in LSTM_INPUT_COLS_FINAL if f not in ALL_FINAL_COLS]
    missing_static = [f for f in STATIC_DENSE_COLS_FINAL if f not in ALL_FINAL_COLS]
    
    if missing_lstm:
        print(f"   ⚠️ WARNING: {len(missing_lstm)} LSTM features not found in transformed data:")
        print(f"      {missing_lstm[:5]}...")  # Show first 5
        # Filter out missing features
        LSTM_INPUT_COLS_FINAL = [f for f in LSTM_INPUT_COLS_FINAL if f in ALL_FINAL_COLS]
    
    if missing_static:
        print(f"   ⚠️ WARNING: {len(missing_static)} Static features not found in transformed data:")
        print(f"      {missing_static[:5]}...")
        STATIC_DENSE_COLS_FINAL = [f for f in STATIC_DENSE_COLS_FINAL if f in ALL_FINAL_COLS]
    # ============================================================
    # STEP 7: Print dimensions for verification
    # ============================================================
    print(f"\n   📊 Feature Breakdown:")
    print(f"   LSTM input features: {len(LSTM_INPUT_COLS_FINAL)}")
    print(f"      - Sequential: {len(lstm_sequential_features)}")
    print(f"      - Repayment OHE: {len(lstm_repayment_features)}")
    
    print(f"   Static dense features: {len(STATIC_DENSE_COLS_FINAL)}")
    print(f"      - Numerical: {len(static_numerical_features)}")
    print(f"      - OHE: {len(static_ohe_features)}")
    
    print(f"   Embedding features: {len(STATIC_EMBEDDING_COLS)}")
    
    # Sequence Reshaping
    def reshape_for_prediction(X_df, lstm_cols, static_dense_cols, embedding_cols, max_len):
        grouped = X_df.groupby('LOAN_ID')
        loan_ids = list(grouped.groups.keys())
        
        X_lstm = np.zeros((len(loan_ids), max_len, len(lstm_cols)), dtype=np.float32)
        X_static_dense = np.zeros((len(loan_ids), len(static_dense_cols)), dtype=np.float32)
        X_static_embed = np.zeros((len(loan_ids), len(embedding_cols)), dtype=np.int16)
        y_actual = np.zeros((len(loan_ids),), dtype=np.int8)
        
        for i, loan_id in enumerate(loan_ids):
            loan_data = grouped.get_group(loan_id)
            sequence = loan_data[lstm_cols].values
            
            # Pad or truncate sequence
            if len(sequence) >= max_len:
                X_lstm[i, :, :] = sequence[-max_len:]
            else:
                X_lstm[i, -len(sequence):, :] = sequence
            
            # Get static features from last record
            last_record = loan_data.iloc[-1]
            X_static_dense[i, :] = last_record[static_dense_cols].values
            X_static_embed[i, :] = last_record[embedding_cols].values.astype(np.int16)
            y_actual[i] = last_record['ACTUAL_NEXT_EMI_ISSUE'].astype(np.int8)
        
        return X_lstm, X_static_dense, X_static_embed, loan_ids, y_actual
    
    # Execute reshaping
    print("   Reshaping sequences for model input...")
    X_predict_lstm, X_predict_static_dense, X_predict_static_embed, prediction_loan_ids, y_actual = \
        reshape_for_prediction(X_predict_df, LSTM_INPUT_COLS_FINAL, STATIC_DENSE_COLS_FINAL,
                              STATIC_EMBEDDING_COLS, MAX_SEQUENCE_LENGTH)
    
    print(f"   Reshaped data shapes:")
    print(f"      LSTM: {X_predict_lstm.shape}")
    print(f"      Static Dense: {X_predict_static_dense.shape}")
    print(f"      Embedding: {X_predict_static_embed.shape}")

    print(f"\n   🔍 Dimension Verification:")
    print(f"   From training notebook, your shapes should be:")
    print(f"      LSTM: (n_loans, 8, {12 + len(REPAYMENT_CAT_OHE_COLS)})")  # 12 sequential + OHE cols
    print(f"      Static Dense: (n_loans, {6 + len(static_ohe_features)})")  # 6 numerical + OHE
    print(f"      Embedding: (n_loans, 2)")

    # Expected dimensions (get these from your training output)
    EXPECTED_LSTM_FEATURES = 12 + len(REPAYMENT_CAT_OHE_COLS)  # Sequential + Repayment OHE
    EXPECTED_STATIC_FEATURES = 6 + len(static_ohe_features)     # Numerical + State/Marital/Loan_Schedule OHE
    
    if X_predict_lstm.shape[2] != EXPECTED_LSTM_FEATURES:
        print(f"   ❌ LSTM MISMATCH! Expected {EXPECTED_LSTM_FEATURES}, got {X_predict_lstm.shape[2]}")
    else:
        print(f"   ✅ LSTM dimensions match!")
    
    if X_predict_static_dense.shape[1] != EXPECTED_STATIC_FEATURES:
        print(f"   ❌ STATIC MISMATCH! Expected {EXPECTED_STATIC_FEATURES}, got {X_predict_static_dense.shape[1]}")
    else:
        print(f"   ✅ Static Dense dimensions match!")

    
    # Model Inference
    print("\n   Running model inference...")
    prediction_inputs = {
        'lstm_input': X_predict_lstm,
        'static_dense_input': X_predict_static_dense,
        'embedding_input': X_predict_static_embed
    }
    
    y_pred_proba = model.predict(prediction_inputs, verbose=0)[:, 0]
    y_pred_class = (y_pred_proba > 0.5).astype(np.int8)
    print(f"✅ Prediction complete for {len(prediction_loan_ids)} unique loans.")
    return prediction_loan_ids, y_pred_proba, y_pred_class, y_actual, prediction_inputs, LSTM_INPUT_COLS_FINAL, STATIC_DENSE_COLS_FINAL

# ===================================================================================
# PART 5: EVALUATION AND REPORTING
# ===================================================================================

def evaluate_performance(y_true, y_pred_class, y_pred_proba):
    """
    Generates a professional performance report for the back-test.
    """
    print("\n" + "="*60)
    print("📈 BACK-TEST PERFORMANCE REPORT")
    print("="*60)
    
    acc = accuracy_score(y_true, y_pred_class)
    cm = confusion_matrix(y_true, y_pred_class)
    
    print(f"Overall Accuracy: {acc:.4f}")
    print("\nConfusion Matrix:")
    print(pd.DataFrame(cm, 
                       index=['Actual Paid', 'Actual Default'], 
                       columns=['Pred Paid', 'Pred Default']))
    
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred_class, target_names=['Paid (0)', 'Not Paid (1)']))
    
    # Senior insight: Financial models care about the "Cost of False Negatives"
    fn = cm[1, 0]
    print(f"⚠️  Missed Defaults (False Negatives): {fn}")
    print("="*60 + "\n")


def calculate_feature_importance_corrected(model, inputs_dict, y_true, y_pred_proba_baseline, 
                                           lstm_cols, static_cols, embed_cols):
    """
    Calculates feature importance using DROP-COLUMN method with AUC metric.
    This matches the training methodology exactly.
    """
    from sklearn.metrics import roc_auc_score
    
    print("\n" + "="*70)
    print("📊 FEATURE IMPORTANCE CALCULATION (Drop-Column Method with AUC)")
    print("="*70)
    
    # Calculate baseline AUC
    baseline_auc = roc_auc_score(y_true, y_pred_proba_baseline)
    print(f"Baseline Test AUC: {baseline_auc:.4f}\n")
    
    importance_results = []
    
    # === LSTM Features (Sequential) ===
    print("Calculating importance for LSTM features...")
    for i, col_name in enumerate(lstm_cols):
        # Create corrupted copy
        inputs_corrupted = {k: v.copy() for k, v in inputs_dict.items()}
        
        # ✅ FIXED: Zero out the feature across all timesteps (matching training)
        inputs_corrupted['lstm_input'][:, :, i] = 0
        
        # Predict with corrupted data
        y_pred_corrupted = model.predict(inputs_corrupted, verbose=0)[:, 0]
        corrupted_auc = roc_auc_score(y_true, y_pred_corrupted)
        
        # Calculate importance as AUC drop
        importance = baseline_auc - corrupted_auc
        
        importance_results.append({
            'Feature': col_name.replace('num__', ''),
            'Importance': importance,
            'Type': 'Sequential (LSTM)',
            'Baseline_AUC': baseline_auc,
            'Corrupted_AUC': corrupted_auc
        })
        
        if i % 5 == 0:
            print(f"   Processed {i+1}/{len(lstm_cols)} LSTM features...")
    
    # === Static Dense Features ===
    print("\nCalculating importance for Static Dense features...")
    for i, col_name in enumerate(static_cols):
        inputs_corrupted = {k: v.copy() for k, v in inputs_dict.items()}
        
        # ✅ FIXED: Zero out the feature
        inputs_corrupted['static_dense_input'][:, i] = 0
        
        y_pred_corrupted = model.predict(inputs_corrupted, verbose=0)[:, 0]
        corrupted_auc = roc_auc_score(y_true, y_pred_corrupted)
        
        importance = baseline_auc - corrupted_auc
        
        # Determine if it's OHE or numerical
        feature_type = 'Static (OHE)' if col_name.startswith('cat__') else 'Static (Numerical)'
        
        importance_results.append({
            'Feature': col_name.replace('num__', '').replace('cat__', ''),
            'Importance': importance,
            'Type': feature_type,
            'Baseline_AUC': baseline_auc,
            'Corrupted_AUC': corrupted_auc
        })
    
    # === Embedding Features ===
    print("\nCalculating importance for Embedding features...")
    for i, col_name in enumerate(embed_cols):
        inputs_corrupted = {k: v.copy() for k, v in inputs_dict.items()}
        
        # ✅ FIXED: Set to 0 (unknown category)
        inputs_corrupted['embedding_input'][:, i] = 0
        
        y_pred_corrupted = model.predict(inputs_corrupted, verbose=0)[:, 0]
        corrupted_auc = roc_auc_score(y_true, y_pred_corrupted)
        
        importance = baseline_auc - corrupted_auc
        
        importance_results.append({
            'Feature': col_name.replace('_ENCODED', ''),
            'Importance': importance,
            'Type': 'Static (Embedding)',
            'Baseline_AUC': baseline_auc,
            'Corrupted_AUC': corrupted_auc
        })
    
    # Create DataFrame and sort
    importance_df = pd.DataFrame(importance_results)
    importance_df = importance_df.sort_values('Importance', ascending=False).reset_index(drop=True)
    
    print("\n" + "="*70)
    print("✅ Feature Importance Calculation Complete")
    print("="*70)
    
    return importance_df


# ===================================================================================
# MAIN EXECUTION PIPELINE
# ===================================================================================

if __name__ == "__main__":
    try:
        # 1. Load Model and Artifacts
        preprocessor, model, category_mappings = load_artifacts()

        # 2. Extract Static Data (LOS)
        df_los = load_and_preprocess_los()

        # 3. Extract Sequential Data (LMS)
        df_lms_full, df_actual_targets = load_and_engineer_lms()

        # 4. Merge and Align for Production
        df_final = merge_and_finalize_data(df_los, df_lms_full, df_actual_targets, category_mappings)

        # 5. Inference (✅ Now returns additional data for feature importance)
        loan_ids, probabilities, classes, actuals, prediction_inputs, lstm_cols, static_cols = \
            preprocess_and_predict(df_final, preprocessor, model)

        # 6. Performance Evaluation (Back-testing)
        evaluate_performance(actuals, classes, probabilities)

        # 7. Consolidated Results DataFrame
        results_df = pd.DataFrame({
            'LOAN_ID': loan_ids,
            'ACTUAL_VALUE': actuals,
            'PREDICTED_CLASS': classes,
            'DEFAULT_PROBABILITY': probabilities
        })
        
        # Mapping for readability
        results_df['PREDICTED_STATUS'] = results_df['PREDICTED_CLASS'].map(TWO_CLASS_STATUS_MAP)
        results_df['ACTUAL_STATUS'] = results_df['ACTUAL_VALUE'].map(TWO_CLASS_STATUS_MAP)

        # 8. Save results
        output_file = "oct_2025_backtest_results.csv"
        results_df.to_csv(output_file, index=False)
        print(f"✅ Results successfully exported to: {output_file}")
        
        # ================================================================
        # 9. ✅ CORRECTED FEATURE IMPORTANCE ANALYSIS
        # ================================================================
        
        print("\n" + "🔍"*35)
        print("STARTING FEATURE IMPORTANCE ANALYSIS")
        print("🔍"*35 + "\n")
        
        # Calculate feature importance with corrected method
        importance_df = calculate_feature_importance_corrected(
            model=model,
            inputs_dict=prediction_inputs,
            y_true=actuals,
            y_pred_proba_baseline=probabilities,
            lstm_cols=lstm_cols,
            static_cols=static_cols,
            embed_cols=STATIC_EMBEDDING_COLS
        )
        
        # Display Top 20 Features
        print("\n" + "="*70)
        print("🏆 TOP 20 MOST IMPORTANT FEATURES")
        print("="*70)
        print(importance_df[['Feature', 'Type', 'Importance', 'Corrupted_AUC']].head(20).to_string(index=False))
        
        # Save importance results
        importance_df.to_csv('feature_importance_results.csv', index=False)
        print(f"\n✅ Feature importance saved to: feature_importance_results.csv")
        
        # ✅ CORRECTED VISUALIZATION
        print("\n📊 Generating feature importance visualization...")
        
        plt.figure(figsize=(12, 8))
        
        # Get top 20 features
        top_features = importance_df.head(20)
        
        # Create color map based on feature type
        type_colors = {
            'Sequential (LSTM)': '#2E86AB',
            'Static (Numerical)': '#A23B72',
            'Static (OHE)': '#F18F01',
            'Static (Embedding)': '#C73E1D'
        }
        colors = [type_colors.get(t, '#808080') for t in top_features['Type']]
        
        # Create horizontal bar plot
        bars = plt.barh(range(len(top_features)), top_features['Importance'], color=colors)
        plt.yticks(range(len(top_features)), top_features['Feature'])
        plt.xlabel('Importance (AUC Drop when Feature Removed)', fontsize=12, fontweight='bold')
        plt.title('Top 20 Feature Importance - EMI Default Prediction\n(Drop-Column Method with AUC Metric)', 
                  fontsize=14, fontweight='bold', pad=20)
        plt.gca().invert_yaxis()
        
        # Add legend
        from matplotlib.patches import Patch
        legend_elements = [Patch(facecolor=color, label=label) 
                          for label, color in type_colors.items()]
        plt.legend(handles=legend_elements, loc='lower right', frameon=True, shadow=True)
        
        # Add grid for better readability
        plt.grid(axis='x', alpha=0.3, linestyle='--')
        plt.tight_layout()
        
        # Save plot
        plt.savefig('feature_importance_plot.png', dpi=300, bbox_inches='tight')
        print("✅ Feature importance plot saved to: feature_importance_plot.png")
        plt.show()
        
        # ✅ ADDITIONAL ANALYSIS: Feature importance by type
        print("\n" + "="*70)
        print("📈 FEATURE IMPORTANCE SUMMARY BY TYPE")
        print("="*70)
        type_summary = importance_df.groupby('Type')['Importance'].agg(['count', 'mean', 'sum', 'max'])
        type_summary.columns = ['Count', 'Avg Importance', 'Total Importance', 'Max Importance']
        print(type_summary.to_string())
        
        print("\n" + "="*70)
        print("✅ PIPELINE EXECUTION COMPLETE")
        print("="*70 + "\n")
        
    except Exception as e:
        print(f"\n❌ PIPELINE FAILED: {str(e)}")
        import traceback
        traceback.print_exc()


✅ Success: Preprocessor, Model, and Mappings loaded.
--- Loading LOS Data from PostgreSQL Table: Ashirvad ---
✅ LOS data loaded. Shape: (1111834, 52)
--- Applying LOS Preprocessing (Matching Training) ---
   Imputed TOTAL_INCOME with value: 20000.00
   Imputed TOTAL_EXPENSE with value: 1760.00
✅ LOS Preprocessing Complete. Shape: (1095563, 16)
--- Loading 24 LMS Excel file(s) ---
   ✅ Loaded: test_001.xlsx
   ✅ Loaded: test_002.xlsx
   ✅ Loaded: test_003.xlsx
   ✅ Loaded: test_004.xlsx
   ✅ Loaded: test_005.xlsx
   ✅ Loaded: test_006.xlsx
   ✅ Loaded: test_007.xlsx
   ✅ Loaded: test_008.xlsx
   ✅ Loaded: test_009.xlsx
   ✅ Loaded: test_010.xlsx
   ✅ Loaded: test_011.xlsx
   ✅ Loaded: test_012.xlsx
   ✅ Loaded: test_013.xlsx
   ✅ Loaded: test_014.xlsx
   ✅ Loaded: test_015.xlsx
   ✅ Loaded: test_016.xlsx
   ✅ Loaded: test_017.xlsx
   ✅ Loaded: test_018.xlsx
   ✅ Loaded: test_019.xlsx
   ✅ Loaded: test_020.xlsx
   ✅ Loaded: test_021.xlsx
   ✅ Loaded: test_022.xlsx
   ✅ Loaded: test_023.

Traceback (most recent call last):
  File "C:\Users\101032\AppData\Local\Temp\ipykernel_12472\719006584.py", line 648, in <module>
    df_final = merge_and_finalize_data(df_los, df_lms_full, df_actual_targets, category_mappings)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\101032\AppData\Local\Temp\ipykernel_12472\719006584.py", line 248, in merge_and_finalize_data
    df_combined = pd.merge(df_los, df_lms, on='LOAN_ID', how='inner', suffixes=('_static', '_lms'))
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\VS_CODE_PROJECTS\CollectionEfficiencyMFI_LOS_LMS\env_local_311\Lib\site-packages\pandas\core\reshape\merge.py", line 183, in merge
    return op.get_result(copy=copy)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\VS_CODE_PROJECTS\CollectionEfficiencyMFI_LOS_LMS\env_local_311\Lib\site-packages\pandas\core\reshape\merge.py", line 885, in get_result
 

In [11]:
STATIC_COLS_NUMERICAL

['TOTAL_INCOME', 'TOTAL_EXPENSE', 'LOAN_AMOUNT', 'AGE', 'CYCLE']